In [1]:
# 데이터 전처리를 추가적으로 수행합니다.

# 1. 영어는 모두 소문자로 변경
# 2. 특수문자 처리 등을 확인합니다 - 인호님의 morphological_analysis_okt와 방식 맞춰서
# 3. "~하신 분, ~해요" 등의 영향을 줄 수 있는 요소들을 삭제한 후, 다시 연결합니다.

In [2]:
import pandas as pd
import numpy as np
from konlpy.tag import Hannanum,Okt
import re
from collections import Counter
import ast

In [3]:
# 데이터 전처리
def preprocess_data(text):
    # 정규식을 사용하여 특수문자 제거
    text = re.sub(r'[-•·*,\r\n\(\)\[\]\/:ㆍ\<\>＜＞［］]', ' ', text)
    text = text.lower()
    
    # 토큰화
    okt = Okt()
    tokens = okt.pos(text)
    return tokens

In [4]:
# pos 종류 확인
def kind_pos(text):
    text = ast.literal_eval(text)
    kind_pos = [pos for word, pos in text]
    return kind_pos

In [5]:
# 특정 품사의 pos만 선택
def pos_data(text, posname):
    text = ast.literal_eval(text)
    pos_data = [word for word, pos in text if pos in [posname]]
    return str(pos_data)

In [46]:
# # 데이터 로드
# result_csv = pd.read_csv('pre_result.csv')

# # 텍스트 전처리
# columns = ['자격요건','우대조건','복지','회사소개','주요업무']

# for column in columns:
#     result_csv[column] = result_csv[column].apply(preprocess_data)

# result_csv.to_csv("pre_result_okt.csv",index=False)
# print(len(result_csv))
# result_csv.head(1)


8512


,회사명,공고명,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무,직무내용
0,(유)미소,Back-end Developer,"[(이런, Adjective), (분과, Noun), (함께하고, Verb), (싶...","[(이런, Adjective), (분이라면, Verb), (더, Noun), (좋아...",3,https://www.jobplanet.co.kr/job/search?posting...,"[(보상, Noun), (및, Noun), (혜택, Noun), (정직, Noun)...","[(mission, Alpha), (‘, Foreign), (we, Alpha), ...","[(미소, Noun), (프로덕트팀, Noun), (이, Josa), (일, Nou...",서버/백엔드 개발


In [6]:
# 품사 전체 종류 확인
result_csv = pd.read_csv('pre_result_okt.csv')
result_csv1 = result_csv.copy()
result_csv1.head(1)
columns = ['자격요건','우대조건','복지','회사소개','주요업무']
for column in columns:
    result_csv1[column] = result_csv[column].apply(kind_pos)
result_csv1.head(1)

,회사명,공고명,자격요건,우대조건,경력조건,출처 URL,복지,회사소개,주요업무,직무내용
0,(유)미소,Back-end Developer,"[Adjective, Noun, Verb, Verb, Noun, Noun, Noun...","[Adjective, Verb, Noun, Adjective, Noun, Noun,...",3,https://www.jobplanet.co.kr/job/search?posting...,"[Noun, Noun, Noun, Noun, Suffix, Alpha, Alpha,...","[Alpha, Foreign, Alpha, Alpha, Alpha, Alpha, A...","[Noun, Noun, Josa, Noun, Verb, Noun, Number, P...",서버/백엔드 개발


In [7]:
pos_df = result_csv1[['자격요건','우대조건','복지','회사소개','주요업무']]
poses = pos_df.values.flatten().tolist()

pos_list = [element for sublist in poses for element in sublist]
pos_list

counted_values = Counter(pos_list)
counted_values

Counter({'Noun': 2263884,
         'Josa': 599793,
         'Alpha': 368241,
         'Verb': 266416,
         'Adjective': 202705,
         'Punctuation': 161387,
         'Number': 111269,
         'Suffix': 84796,
         'Modifier': 42278,
         'Foreign': 33236,
         'Adverb': 29046,
         'Determiner': 3894,
         'URL': 3735,
         'Hashtag': 1988,
         'Exclamation': 1811,
         'Conjunction': 1575,
         'VerbPrefix': 904,
         'KoreanParticle': 142,
         'Eomi': 103,
         'ScreenName': 44,
         'Email': 36,
         'CashTag': 13})

In [8]:
# 가장 적게 등장한 품사부터 하나씩 필요없는 품사를 확인해서 날려버릴 품사 / 남길 품사 결정하는 함수 정의
def check_pos_data(posname): # 여러 개 선택 시 posname='Verb','Adjective'
    result_csv2 = result_csv.copy()
    for column in columns:
        result_csv2[column] = result_csv[column].apply(lambda x: pos_data(text=x, posname=posname)) 

    pos_df = result_csv2[['자격요건','우대조건','복지','회사소개','주요업무']]
    poses = pos_df.values.flatten().tolist()
    counted_values = Counter([element for sublist in poses for element in eval(sublist)])
    return counted_values

In [9]:
check_pos_data('CashTag')

Counter({'$spot': 13})

In [10]:
check_pos_data('Email')

Counter({'start@ridi.com': 11,
         'recruitingops@coupang.com': 11,
         'stadia@google': 3,
         'genie.hr@kt.com': 2,
         'career@clo3d.com': 2,
         'wa_iri@naver.com': 1,
         'recruit@deali.net': 1,
         'contact@art': 1,
         'lambda@edge': 1,
         'contact@mxncommerce.com': 1,
         'interline30@naver.com': 1,
         'recruit@pluto.im': 1})

In [11]:
check_pos_data('ScreenName')

Counter({'@redbrick': 12,
         '@lgvehiclecomponentso': 8,
         '@tsnlab': 7,
         '@lenditkr': 4,
         '@sungjoonkim': 4,
         '@stelsiofficial': 1,
         '@testing': 1,
         '@boosters': 1,
         '@b2b': 1,
         '@kakaoventures': 1,
         '@stpcautonomousrobot2': 1,
         '@zoidswild': 1,
         '@cocone_m': 1,
         '@albertmamy': 1})

In [12]:
check_pos_data('Eomi')

Counter({'하오니': 23,
         '한데': 16,
         '니다': 15,
         '습니다': 14,
         '기에': 6,
         '기로': 5,
         '하듯': 5,
         '어': 4,
         '거야': 4,
         '우면': 3,
         '데도': 3,
         '세요': 3,
         '다면': 2})

In [13]:
check_pos_data('KoreanParticle')

Counter({'ㄴ': 67,
         'ㅣ': 18,
         'ㅁ': 15,
         'ㅠㅠ': 10,
         'ㅡ': 8,
         'ㅋㅌ': 7,
         'ㅇ': 5,
         'ㅠ': 4,
         'ㅍ': 3,
         'ㅏ': 1,
         'ㄷㄷㄷㄷㄷ': 1,
         'ㄱㄱ': 1,
         'ㅋㅋ': 1,
         'ㅅ': 1})

In [14]:
check_pos_data('VerbPrefix')

Counter({'안': 271, '잘': 231, '내': 216, '못': 184, '짓': 1, '엇': 1})

In [15]:
check_pos_data('Conjunction')

Counter({'그리고': 1184,
         '하지만': 303,
         '그러나': 44,
         '그렇다고': 16,
         '그런데': 12,
         '그렇지만': 6,
         '그러기에': 5,
         '그러므로': 3,
         '그러다가': 1,
         '그리하여': 1})

In [16]:
check_pos_data('Exclamation')

Counter({'플': 523,
         '아': 439,
         '하': 286,
         '휴': 222,
         '야': 183,
         '엑': 113,
         '피': 16,
         '에그': 12,
         '어와': 7,
         '아이코': 5,
         '후후': 2,
         '아잉': 2,
         '호에': 1})

In [17]:
check_pos_data('Hashtag')

Counter({'#회사': 50,
         '#wemix': 42,
         '#인센티브': 39,
         '#건강검진': 37,
         '#유연근무제': 35,
         '#수평적': 34,
         '#팬플러스': 32,
         '#지원': 32,
         '#맛있는간식냠냠': 32,
         '#맥북으로개발': 32,
         '#자유복장': 32,
         '#반바지': 32,
         '#도서구입비지원': 32,
         '#택시비지원': 32,
         '#ams': 25,
         '#블록체인': 25,
         '#자율복장': 22,
         '#미르4': 21,
         '#나이트크로우': 21,
         '#nft': 21,
         '#dao': 21,
         '#defi': 21,
         '#wemixplay': 21,
         '#nile': 21,
         '#중식제공': 20,
         '#단체보험': 20,
         '#사내카페': 20,
         '#수평적조직': 19,
         '#도서구매비': 16,
         '#팬플러스에': 16,
         '#유연근무': 16,
         '#장기근속휴가': 16,
         '#리프레쉬휴가': 15,
         '#재택근무': 15,
         '#스타트업': 15,
         '#성과급': 14,
         '#휴게실': 11,
         '#조식제공': 11,
         '#복지카드': 11,
         '#요가룸': 11,
         '#수면실': 11,
         '#수유실': 11,
         '#스포츠센터': 11,
         '#휴양시설제휴': 11,
         '#장기근속포상':

In [18]:
check_pos_data('URL')

Counter({'n.news.naver.com': 299,
         'www.youtube.com': 243,
         'news.naver.com': 185,
         'tinyurl.com': 110,
         'www.etnews.com': 106,
         'www.hankyung.com': 100,
         'www.mk.co.kr': 90,
         'it.chosun.com': 85,
         'asp.net': 83,
         'blog.naver.com': 76,
         'www.venturesquare.net': 73,
         'www.fnnews.com': 58,
         'www.clo3d.com': 46,
         'news.heraldcorp.com': 45,
         'news.mt.co.kr': 40,
         'www.epnc.co.kr': 40,
         'www.notion.so': 36,
         'www.marvelousdesigner.com': 35,
         'byline.network': 35,
         'www.industrynews.co.kr': 33,
         'www.ksilbo.co.kr': 32,
         'www.nocutnews.co.kr': 30,
         'www.wanted.co.kr': 27,
         'www.techm.kr': 27,
         'github.com': 26,
         'zdnet.co.kr': 23,
         'set.com': 23,
         'www.mobiinside.co.kr': 21,
         'medium.com': 20,
         'jmagazine.joins.com': 20,
         'vb.net': 20,
         'www.ikld.kr

In [19]:
check_pos_data('Determiner')

Counter({'이': 1486,
         '한': 793,
         '저': 485,
         '그': 377,
         '내': 324,
         '두': 304,
         '네': 124,
         '다섯': 1})

In [20]:
check_pos_data('Adverb')

Counter({'함께': 11673,
         '또는': 4911,
         '혹은': 2210,
         '없이': 1768,
         '같이': 795,
         '다': 731,
         '매주': 585,
         '물론': 506,
         '이렇게': 476,
         '열심히': 462,
         '특히': 439,
         '이미': 336,
         '아낌없이': 309,
         '언제나': 269,
         '아직': 260,
         '많이': 249,
         '그래서': 217,
         '오래': 175,
         '히': 163,
         '우': 151,
         '관계없이': 147,
         '오랜': 142,
         '어느': 139,
         '너무': 91,
         '빨리': 86,
         '풍': 85,
         '수시로': 78,
         '매해': 75,
         '오래도록': 73,
         '훨씬': 65,
         '따로': 53,
         '그만': 51,
         '너무나': 49,
         '그렇게': 46,
         '오로지': 45,
         '자유로이': 42,
         '결국': 40,
         '오랫동안': 38,
         '그중': 37,
         '그만큼': 36,
         '씩': 36,
         '아무리': 34,
         '그저': 31,
         '딱': 29,
         '너무나도': 28,
         '씽씽': 28,
         '앙': 27,
         '아직도': 26,
         '되게': 25,
         '적어도': 25,
     

In [21]:
check_pos_data('Foreign')

Counter({'‘': 3801,
         '“': 1976,
         '”': 1904,
         '■': 1658,
         '에': 1194,
         '의': 1138,
         '※': 968,
         '을': 660,
         '→': 583,
         '∙': 580,
         '【': 518,
         '】': 514,
         '▶': 500,
         '까지': 457,
         '차': 445,
         '에는': 423,
         '마다': 409,
         '제': 394,
         '부터': 386,
         '이상': 377,
         '간': 328,
         '・': 293,
         '▷': 278,
         '㈜': 254,
         '\u202f': 239,
         '–': 202,
         '명': 197,
         '◆': 193,
         '✔': 147,
         '👉': 142,
         '♥': 133,
         '、': 127,
         '도': 113,
         '◎': 112,
         '﹒': 109,
         '\ufeff': 99,
         '●': 98,
         '︎': 95,
         '만에': 94,
         '명의': 93,
         '이': 88,
         '건': 87,
         '으로': 86,
         '근무': 85,
         '간의': 84,
         '⦁': 82,
         '年': 81,
         '★': 79,
         '🎁': 78,
         '①': 76,
         '②': 75,
         '✓': 72,
   

In [22]:
check_pos_data('Modifier')

Counter({'경': 4552,
         '반': 3310,
         '근': 2272,
         '대': 2270,
         '전': 1988,
         '지': 1965,
         '조': 1699,
         '사': 1230,
         '연': 1207,
         '오': 962,
         '신': 913,
         '주': 889,
         '즈': 865,
         '문': 829,
         '고': 819,
         '제': 765,
         '속': 753,
         '일': 746,
         '원': 729,
         '영': 708,
         '기': 631,
         '동': 605,
         '서': 593,
         '공': 587,
         '구': 566,
         '수': 552,
         '시': 509,
         '성': 490,
         '타': 431,
         '소': 413,
         '언': 402,
         '세': 344,
         '주요': 294,
         '온': 287,
         '여': 287,
         '모': 277,
         '귀': 248,
         '니': 225,
         '매': 221,
         '일이': 220,
         '현': 219,
         '너': 194,
         '백': 181,
         '극': 178,
         '만': 176,
         '한': 176,
         '천': 166,
         '석': 153,
         '본': 146,
         '한다는': 142,
         '요': 121,
         '삼': 120,

In [23]:
check_pos_data('Suffix')

Counter({'적': 25878,
         '들': 16992,
         '자': 13981,
         '화': 3775,
         '성': 3340,
         '형': 2531,
         '원': 2093,
         '력': 1791,
         '어': 1454,
         '님': 1287,
         '군': 1154,
         '상': 1039,
         '하': 953,
         '간': 919,
         '중': 829,
         '권': 755,
         '장': 659,
         '분들': 651,
         '급': 592,
         '식': 421,
         '계': 385,
         '뿐': 354,
         '이': 335,
         '인': 320,
         '대': 272,
         '씩': 222,
         '분': 206,
         '십': 165,
         '샵': 158,
         '번째': 154,
         '틱': 145,
         '씨': 120,
         '당': 113,
         '언니': 113,
         '측': 88,
         '극': 87,
         '명': 84,
         '째': 82,
         '백': 51,
         '닷컴': 50,
         '개': 39,
         '왕': 33,
         '천': 21,
         '룩': 20,
         '쯤': 18,
         '조': 17,
         '장이': 16,
         '자리': 13,
         '양': 7,
         '나라': 5,
         '쟁이': 4,
         '만': 2,
         '키

In [24]:
check_pos_data('Number')

Counter({'1': 9816,
         '2': 8434,
         '3': 8081,
         '4': 4390,
         '3년': 3776,
         '5': 2896,
         '5년': 2120,
         '1회': 1915,
         '6': 1767,
         '2년': 1714,
         '1년': 1463,
         '8': 1162,
         '10': 1123,
         '2022년': 1069,
         '10시': 995,
         '7': 994,
         '5일': 946,
         '100만원': 872,
         '10년': 859,
         '100%': 848,
         '9': 799,
         '00': 769,
         '1시간': 751,
         '2021년': 751,
         '4년': 730,
         '2023년': 635,
         '2시간': 620,
         '000': 613,
         '2020년': 607,
         '2회': 581,
         '30': 579,
         '8시': 564,
         '10만원': 526,
         '7년': 512,
         '2021': 504,
         '11시': 489,
         '30분': 486,
         '2019년': 469,
         '1일': 461,
         '000원': 448,
         '12': 431,
         '20만원': 423,
         '200만원': 420,
         '2022': 412,
         '2018년': 385,
         '20': 377,
         '10일': 365,
         '3

In [25]:
check_pos_data('Punctuation')

Counter({'.': 108486,
         '!': 10174,
         "'": 7905,
         '"': 6270,
         '&': 5249,
         '’': 3718,
         '?': 3110,
         '++': 2904,
         '~': 2665,
         '+': 2331,
         '=': 2296,
         '#': 1830,
         '_': 642,
         '|': 555,
         '`': 460,
         '…': 369,
         '."': 340,
         '!!': 310,
         '...': 252,
         '%': 218,
         '..': 215,
         '!"': 85,
         '###': 84,
         ';': 67,
         "?'": 62,
         '##': 60,
         '?!': 55,
         '?"': 48,
         '#.': 47,
         "'~": 39,
         '~!': 36,
         '!!!': 31,
         "#'": 31,
         '^^': 27,
         ".'": 20,
         "!'": 20,
         '..."': 15,
         '+@': 14,
         '"~': 13,
         "''": 13,
         '.’': 11,
         '....': 11,
         '!’': 10,
         '==': 10,
         '#’': 10,
         "'.": 9,
         '?’': 9,
         '~!!': 9,
         '{': 9,
         '}': 9,
         '!!!!!': 9,
         

In [31]:
check_pos_data('Adjective')
# Adjective not in..
['있습니다','있는','있으신','입니다','있도록','있어요','있으며', '있고','있게',
 '이런', '이러한','좋아요','있다면','부탁드립니다','있어','있을','있거나','있으','있으면',
 '있나요','있답니다','유치하였습니다','기대합니다','유치하였으며','있죠','있기에','유치하며',
 '있지', '있으니', '유치했으며', '있던','있으시다면','원해요',]

Counter({'있습니다': 22426,
         '있는': 19077,
         '있으신': 13034,
         '입니다': 10675,
         '다양한': 9804,
         '있도록': 4843,
         '새로운': 4759,
         '있어요': 4714,
         '있으며': 3665,
         '자유로운': 3348,
         '필요한': 3330,
         '자유롭게': 2455,
         '가능한': 2311,
         '많은': 2233,
         '높은': 2225,
         '빠르게': 2208,
         '가능하신': 2005,
         '이런': 1964,
         '좋은': 1933,
         '없는': 1927,
         '아니라': 1513,
         '원활한': 1448,
         '같은': 1399,
         '성장하고': 1357,
         '원하는': 1300,
         '있고': 1249,
         '성장할': 1033,
         '빠른': 989,
         '아닌': 915,
         '성장하는': 906,
         '있게': 753,
         '가능합니다': 752,
         '능숙하신': 750,
         '어떤': 735,
         '건강한': 717,
         '이러한': 716,
         '뛰어난': 694,
         '좋아요': 687,
         '유연한': 683,
         '끊임없이': 667,
         '쾌적한': 650,
         '즐거운': 609,
         '높으신': 589,
         '계신': 568,
         '익숙하신': 564,
         '깊은': 553,
      

In [27]:
check_pos_data('Verb')

Counter({'합니다': 23709,
         '하는': 20155,
         '할': 15268,
         '하여': 9500,
         '해': 6520,
         '된': 6287,
         '하며': 5481,
         '해요': 4681,
         '하기': 3628,
         '하신': 3603,
         '는': 3452,
         '될': 3328,
         '따라': 2876,
         '되는': 2613,
         '와': 2545,
         '하고': 2345,
         '드려요': 2089,
         '한': 2034,
         '됩니다': 2010,
         '만들어': 1969,
         '되어': 1877,
         '하실': 1687,
         '이를': 1626,
         '하시는': 1473,
         '드립니다': 1462,
         '따른': 1394,
         '하게': 1381,
         '잘': 1325,
         '가': 1320,
         '하지': 1312,
         '찾고': 1291,
         '가진': 1286,
         '하고자': 1284,
         '찾습니다': 1090,
         '하세요': 1083,
         '해드려요': 1014,
         '대': 1002,
         '본': 997,
         '만들고': 990,
         '않고': 988,
         '싶은': 965,
         '해주세요': 939,
         '만드는': 909,
         '나가고': 902,
         '큰': 878,
         '비전': 838,
         '나갈': 821,
         '해서':

In [28]:
check_pos_data('Alpha')

Counter({'c': 7365,
         'ai': 5912,
         'aws': 5610,
         'react': 4937,
         'js': 4661,
         'java': 4159,
         'python': 4118,
         'api': 4086,
         'https': 3923,
         'javascript': 3469,
         'it': 3286,
         'git': 3195,
         'spring': 3193,
         'b': 2852,
         'linux': 2630,
         'html': 2558,
         'typescript': 2453,
         'd': 2364,
         'a': 2290,
         'mysql': 2250,
         'ios': 2089,
         'android': 2072,
         'and': 2038,
         'framework': 1990,
         'node': 1783,
         'sw': 1765,
         'docker': 1740,
         'data': 1736,
         'sql': 1727,
         'github': 1682,
         'to': 1679,
         'css': 1671,
         'ui': 1578,
         'web': 1507,
         'the': 1464,
         'cloud': 1399,
         'db': 1383,
         's': 1362,
         'nan': 1340,
         'kotlin': 1335,
         'native': 1278,
         'kubernetes': 1222,
         'vue': 1209,
        

In [29]:
check_pos_data('Josa')

Counter({'을': 105702,
         '의': 65612,
         '를': 61865,
         '에': 54898,
         '이': 42021,
         '하고': 26435,
         '으로': 26256,
         '한': 25693,
         '과': 24898,
         '는': 18107,
         '가': 17525,
         '와': 15544,
         '인': 14842,
         '에서': 14516,
         '은': 14440,
         '로': 12731,
         '도': 4360,
         '에게': 3791,
         '고': 3679,
         '까지': 3480,
         '서': 2134,
         '나': 1961,
         '에는': 1803,
         '부터': 1773,
         '이고': 1753,
         '요': 1558,
         '에도': 1364,
         '만': 1349,
         '이며': 1225,
         '과의': 1161,
         '에서의': 1040,
         '다': 1034,
         '지': 946,
         '이나': 898,
         '보다': 885,
         '이라는': 874,
         '에서는': 849,
         '와의': 808,
         '라는': 724,
         '이신': 718,
         '면': 713,
         '만의': 685,
         '며': 577,
         '엔': 557,
         '에서도': 555,
         '여': 531,
         '든': 529,
         '으로써': 526,
         '라'

In [30]:
check_pos_data('Noun')

Counter({'개발': 47352,
         '및': 46480,
         '지원': 44078,
         '분': 41527,
         '경험': 40405,
         '등': 26123,
         '서비스': 25232,
         '수': 23379,
         '제공': 19753,
         '운영': 18779,
         '기술': 17413,
         '업무': 16365,
         '사용': 13448,
         '대한': 13121,
         '위': 12988,
         '휴가': 11759,
         '기반': 11380,
         '이상': 11223,
         '관련': 10850,
         '데이터': 10693,
         '지급': 10172,
         '팀': 9782,
         '환경': 9642,
         '관리': 9491,
         '를': 9400,
         '비': 9262,
         '플랫폼': 8667,
         '시스템': 8545,
         '기업': 8226,
         '경력': 8125,
         '성장': 7828,
         '일': 7586,
         '위해': 7478,
         '근무': 7438,
         '통해': 7060,
         '보유': 6974,
         '설계': 6971,
         '회사': 6919,
         '이해': 6789,
         '문화': 6307,
         '구축': 6296,
         '솔루션': 6262,
         '분석': 6248,
         '활용': 6183,
         '고객': 6158,
         '교육': 6023,
         '진행': 59

In [ ]:
# 주어진 데이터 파싱하고 pos에 따라 다른 조건 적용 후, 원래 모습으로 반환하여 출력
def pos_selection_josa(data_str):
    data_list = ast.literal_eval(data_str)
    selected_data = []
    for word, pos in data_list:
        if pos in ['Hashtag', 'Punctuation', 'Adjective', 'Alpha', 'Noun', 
                   'Josa', 'VerbPrefix'] and word not in ['분과', '분']:
            selected_data.append(word)
        elif pos == 'Foreign' and word in ['근무']:
            selected_data.append(word)
        elif pos == 'Verb' and word in ['중요한 것들 확인해서 넣기']:
            selected_data.append(word)
    return ' '.join(selected_data)

In [ ]:
result_csv2 = result_csv.copy()
    for column in columns:
        result_csv2[column] = result_csv[column].apply(pos_selection_josa) 
result_csv2.to_csv("pre_result_josa.csv",index=False)

In [ ]:
# 주어진 데이터 파싱하고 pos에 따라 다른 조건 적용 후, 원래 모습으로 반환하여 출력
def pos_selection(data_str):
    data_list = ast.literal_eval(data_str)
    selected_data = []
    for word, pos in data_list:
        if pos in ['Hashtag', 'Punctuation', 'Adjective', 'Alpha', 'Noun'
                   ] and word not in ['분과', '분']:
            selected_data.append(word)
        elif pos == 'Foreign' and word in ['근무']:
            selected_data.append(word)
        elif pos == 'Verb' and word in ['중요한 것들 확인해서 넣기']:
            selected_data.append(word)
    return ' '.join(selected_data)

In [ ]:
result_csv3 = result_csv.copy()
    for column in columns:
        result_csv3[column] = result_csv[column].apply(pos_selection) 
result_csv3.to_csv("pre_result_done.csv",index=False)

In [ ]:
# 조사 포함, 품사 적당히 남겨놓은 파일 : pre_result_josa.csv
# 품사도 까다롭게 보고, 선택된 품사 중에서도 필요없는 불용어를 제거한 파일 : pre_result_done.csv